## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails. I'm using Pushover for this notebook)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.

Or, use Pushover.

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from typing import Dict
import os
import asyncio
import json
import requests

# Note: This notebook is configured to work with OpenRouter by mapping
# OPENROUTER_API_KEY / OPENROUTER_API_BASE to OpenAI-style env vars.
# Pushover is used for sending notifications/emails (no SendGrid required).


In [2]:
load_dotenv(override=True)

True

In [28]:
# Configure OpenRouter (map to OpenAI-compatible env vars if provided)
# You do NOT need to put the model name in your .env — the notebook sets a default.
if os.getenv("OPENROUTER_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENROUTER_API_KEY")
if os.getenv("OPENROUTER_API_BASE"):
    os.environ["OPENAI_BASE_URL"] = os.getenv("OPENROUTER_API_BASE")

# Default model (set here; no need to add the model to .env)
MODEL = "openai/gpt-5.2-chat"

# Pushover configuration (used for sending notifications/emails)
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

print(f"Using model: {MODEL}")


Pushover user found and starts with u
Pushover token found and starts with a
Using model: openai/gpt-5.2-chat


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file) 

You will see the Pushover verifications, cause i'm using Pushover.

## Step 1: Agent workflow

In [29]:

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails. Only answer with the email, no explanation."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response. Only answer with the email, no explanation"

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails. Only answer with the email, no explanation."


In [30]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=MODEL
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=MODEL
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=MODEL
)


In [ ]:
message = "Write a cold sales email"

result = await Runner.run(sales_agent1, message)
print(result.final_output)

Error getting response: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 65536 tokens, but can only afford 60103. To increase, visit https://openrouter.ai/settings/keys and create a key with a higher total limit', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2q351UzQFtntpyttNwPAz3daWjp'}. (request_id: None)
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************f46c. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 65536 tokens, but can only afford 60103. To increase, visit https://openrouter.ai/settings/keys and create a key with a higher total limit', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2q351UzQFtntpyttNwPAz3daWjp'}

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************f46c. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************f46c. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************f46c. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "

In [10]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Here’s a flexible cold sales email you can adapt. I’ll keep it generic with clear placeholders so you can quickly plug in your details.

---

Subject: Quick idea to improve [specific result] at [Company Name]

Hi [First Name],

I’ve been following [Company Name] and noticed [something specific you observed: recent launch, hiring, growth, news, etc.]. It looks like you’re focused on [goal/initiative relevant to your offer].

I’m reaching out because we help companies like [Similar Company 1] and [Similar Company 2] [achieve a specific outcome], for example:
- [Short, concrete result #1 – e.g., “Reduced support tickets by 27% in 60 days”]  
- [Short, concrete result #2 – e.g., “Increased demo-to-close rate from 18% to 29%”]  

We do this with [your product/service] by:
- [1-sentence explanation of how it works/what it does]  
- [1-sentence explanation of why it’s different or better]  

Would you be open to a brief [15/20]-minute call next week to see if this could help with [relevant go

In [32]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only no explanations. If there are any explanationsor things like: here is a cold sales..., delete them.",
    model=MODEL
)


In [33]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
It looks like you’re collecting variations of the same idea and want a **clean, final version** you can confidently use or customize.

Here’s a **polished, high‑converting cold sales email** that blends the strongest elements from all three and avoids sounding templated or salesy:

---

**Subject:** Quick question about {{Company Name}}

Hi {{First Name}},

I noticed that {{Company Name}} is {{relevant observation—growth, role, tech, initiative}}, and it made me curious how you’re currently handling {{specific pain point}}.

We help {{target role or industry}} {{primary outcome}} by {{what you do differently}}, typically leading to {{specific result or metric}} within {{timeframe}}.

Would it make sense to spend **15 minutes next week** to see whether this could be useful for {{Company Name}}?  
If not, I’m happy to point you in the right direction.

Best,  
{{Your Name}}  
{{Title}} | {{Company}}  
{{Website}} | {{Calendar or Phone}}

---

### Optional tweaks (use sp

Now go and check out the **trace**:

`Openrouter`doesnt have any **traces**

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [34]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=MODEL,
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=MODEL,
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=MODEL,
)


In [35]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails. Only answer with the email, no explanation.', prompt=None, handoffs=[], model='openai/gpt-5.2-chat', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [36]:
import os
import requests
from typing import Dict

PUSHOVER_API_URL = "https://api.pushover.net/1/messages.json"

@function_tool
def send_email(body: str) -> Dict[str, str]:
    """
    Send a notification through the Pushover API.
    """
    payload = {
        "token": os.environ["PUSHOVER_TOKEN"],
        "user": os.environ["PUSHOVER_USER"],
        "title": "Sales Notification",
        "message": body,
    }

    try:
        response = requests.post(
            PUSHOVER_API_URL,
            data=payload,
            timeout=10,
        )
        response.raise_for_status()
        return {"status": "success"}
    except requests.RequestException as error:
        return {
            "status": "error",
            "message": str(error),
        }



### This has automatically been converted into a tool, with the boilerplate json created

In [37]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send a notification through the Pushover API.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB4E3F600>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### And you can also convert an Agent into a tool

In [38]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB778AB60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [11]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB5394C20>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB75039C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', descr

## And now it's time for our Sales Manager - our planning agent

In [41]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model=MODEL)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message) 
    print(result)

    # Send Pushover notification
    if pushover_token and pushover_user and pushover_url:
        import requests
        data = {
            "token": pushover_token,
            "user": pushover_user,
            "message": "Sales email generated successfully!"
        }
        response = requests.post(pushover_url, data=data)
        print(f"Pushover notification sent: {response.status_code}")
    else:
        print("Pushover credentials missing. Notification not sent.")

# Send push notification with Pushover
if pushover_token and pushover_user and pushover_url:
    import requests
    data = {
        "token": pushover_token,
        "user": pushover_user,
        "message": "Sales email generated successfully",
        "title": "ComplAI Sales"
    }
    requests.post(pushover_url, data=data)

    print(result)


RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    ✅ **Cold sales email sent successfully**
    
    Your cold sales email addressed to **“Dear CEO”** has been sent.  
    If you’d like, I can:
    
    - Rewrite it in a different tone (more direct, more casual, more executive-level)
    - Personalize it for a specific industry or company size
    - Optimize it for higher reply rates (subject lines, CTA variations, A/B versions)
    - Shorten it for LinkedIn or follow-up sequences
    
    Just tell me how you’d like to refine it.
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
Pushover notification sent: 200
RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    ✅ **Cold sales email sent successfully**
    
    Your cold sales email addressed to **“Dear CEO”** has been sent.  
    If you’d like, I can:
    
    - Rewrite it in a different to

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

### For people using **Pushover**

*First*, Remember to check the `PUSHOVER_USER` and `PUSHOVER_TOKEN`.

*Second*, Check if the `API Key` of **Pushover** is created.

*Third*, Check if you Saved the **.env**, If none of the 2 before reasons are true, then this one **Should** be.

## Remember to check the **trace**

`OpenRouter` Doesn't have any **traces**.

And then check your **email**!! or **Pushover**!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [42]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=MODEL)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=MODEL)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")



In [43]:
import os
import requests
from typing import Dict

PUSHOVER_API_URL = "https://api.pushover.net/1/messages.json"

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """
    Send a notification with an HTML-formatted message through the Pushover API.
    """
    payload = {
        "token": os.environ["PUSHOVER_TOKEN"],
        "user": os.environ["PUSHOVER_USER"],
        "title": subject,
        "message": html_body,
        "html": 1,
    }

    try:
        response = requests.post(
            PUSHOVER_API_URL,
            data=payload,
            timeout=10,
        )
        response.raise_for_status()
        return {"status": "success"}
    except requests.RequestException as error:
        return {
            "status": "error",
            "message": str(error),
        }


In [44]:
# Let's look at it
tools = [subject_tool, html_tool, send_html_email]

In [45]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB781B880>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB781B100>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=No

In [46]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=MODEL,
    handoff_description="Convert an email to HTML and send it")



### Now we have 3 tools and 1 handoff

In [47]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB778AB60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FFB75039C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', descrip

In [49]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=MODEL)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)


### Remember to check the trace

Again, `Openrouter` Doesn't have any **traces**

And then check your email! or Pushover!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.